In [1]:
from tqdm import tqdm
import matplotlib.pyplot as plt # type: module
import matplotlib.ticker as ticker
from matplotlib import colormaps
from matplotlib.colors import Normalize

import numpy as np
import os, glob
import time
import warnings

from rur.fortranfile import FortranFile
from rur import uri, uhmi, painter, drawer
from rur.sci.photometry import measure_luminosity
from rur.sci.geometry import get_angles, euler_angle
from rur.utool import rotate_data
from scipy.ndimage import gaussian_filter
uri.timer.verbose=1
# from rur.sci.kinematics import f_getpot

from icl_IO import mode2repo, pklsave, pklload
from icl_tool import *
from icl_numba import large_isin, large_isind, isin
from icl_draw import drawsnap, add_scalebar, addtext, MakeSub_nolabel, label_to_in, fancy_axis, circle
import argparse, subprocess
from importlib import reload
import cmasher as cmr
from copy import deepcopy
from multiprocessing import Pool, shared_memory


In [2]:
mode = 'nh'
iout = 1026
repo, rurmode, dp = mode2repo(mode)
snap = uri.RamsesSnapshot(repo, iout, mode=rurmode)
snaps = uri.TimeSeries(snap)
snaps.read_iout_avail()
nout = snaps.iout_avail['iout']
gals = uhmi.HaloMaker.load(snap, galaxy=True, double_precision=dp)
hals = uhmi.HaloMaker.load(snap, galaxy=False, double_precision=dp)
database = f"/home/jeon/MissingSat/database"

[Output 01026] Age (Gyr) : 11.624 / 13.741, z = 0.17149 (a = 0.8536)


In [3]:
lvl1s = hals[ (hals['level']==1) & (hals['mcontam'] < hals['m'])]
len(lvl1s)

13706

In [4]:
def calc_virial(cx,cy,cz, rmax_pkpc, pos_code, m_msol):
    '''
    input:
        cx,cy,cz : center of halo
        star, dm, cell : data
    output:
        rvir : virial radius
        mvir : virial mass
        rvir_code : virial radius in code unit
    '''
    global snap
    # critical density
    H02 = (snap.H0 * 3.24078e-20)**2 # s-2
    G = 6.6743e-11 # N m2 kg-2 = kg m s-2 m2 kg-2 = m3 s-2 kg-1
    rhoc = 3 * H02 /8 /np.pi /G # kg m-3
    rhoc *= 5.02785e-31  * (3.086e+19)**3 # Msol ckpc-3
    rhoc /= (snap.aexp**3) # Msol pkpc-3

    # Sorting
    dis = distance3d(pos_code[:,0], pos_code[:,1], pos_code[:,2], cx, cy, cz)/snap.unit['kpc'] # pkpc
    mask = dis<rmax_pkpc
    argsort = np.argsort(dis[mask])
    dis = dis[mask][argsort] # pkpc
    mas = m_msol[mask][argsort] # Msol

    # Inside density
    cmas = np.cumsum(mas) # Msol
    vols = 4/3*np.pi * dis**3 # pkpc^3
    rhos = cmas / vols # Msol pkpc-3

    arg = np.argmin(np.abs(rhos - 200*rhoc))
    rvir = dis[arg] # pkpc
    if(rvir>=np.max(dis)):
        warnings.warn("rvir is larger than maximum distance!\nEnlarge the box size!")
    elif(rvir<=np.min(dis)):
        warnings.warn("rvir is smaller than minimum distance!\nNot enough particles!")
    else:
        pass
    rvir_code = rvir * snap.unit['kpc'] # code unit
    mvir = cmas[arg] # Msol
    return rvir, mvir, rvir_code

In [5]:
def calc_virial_mp(hal, kwargs):
    cx,cy,cz = hal['x'], hal['y'], hal['z']
    rmax_pkpc = kwargs['rmax_pkpc']
    pos_code = kwargs['pos_code']
    m_msol = kwargs['m_msol']
    return hal['id'], calc_virial(cx,cy,cz, rmax_pkpc, pos_code, m_msol)

In [6]:
snap.cpulist_part

array([], dtype=int32)

In [8]:
snap_star = uri.RamsesSnapshot(repo, iout, mode=rurmode)
snap_dm = uri.RamsesSnapshot(repo, iout, mode=rurmode)
snap_cell = uri.RamsesSnapshot(repo, iout, mode=rurmode)

virials = np.zeros( len(hals), dtype=[("r200kpc","<f8"), ("m200","<f8"), ("r200","<f8")])
uri.timer.verbose=0
for lvl1 in tqdm( lvl1s ):
    if(len(snap_star.cpulist_part)>400)or(len(snap_dm.cpulist_part)>400)or(len(snap_cell.cpulist_part)>400):
        print(f"Clearing cpulist {len(snap_star.cpulist_part)} {len(snap_dm.cpulist_part)} {len(snap_cell.cpulist_part)}")
        snap_star.clear()
        snap_dm.clear()
        snap_cell.clear()
    snap_star.set_box_halo(lvl1, 2, radius_name='r')
    snap_dm.set_box_halo(lvl1, 2, radius_name='r')
    snap_cell.set_box_halo(lvl1, 2, radius_name='r')
    snap_star.get_part(pname='star', target_fields=['x','y','z','m'], nthread=32)
    snap_dm.get_part(pname='dm', target_fields=['x','y','z','m'], nthread=32)
    snap_cell.get_cell(target_fields=['x','y','z','rho','level'], nthread=32)

    pos_star = snap_star.part['pos']; mass_star = snap_star.part['m','Msol']
    pos_dm = snap_dm.part['pos']; mass_dm = snap_dm.part['m','Msol']
    pos_cell = snap_cell.cell['pos']; mass_cell = snap_cell.cell['m','Msol']
    pos_code = np.vstack( (pos_star, pos_dm, pos_cell) )
    mass_msol = np.hstack( (mass_star, mass_dm, mass_cell) )

    r200kpc, m200, r200 = calc_virial(lvl1['x'], lvl1['y'], lvl1['z'], 2*lvl1['r']/snap.unit['kpc'], pos_code, mass_msol)
    virials[lvl1['id']-1]['r200kpc'] = r200kpc
    virials[lvl1['id']-1]['m200'] = m200
    virials[lvl1['id']-1]['r200'] = r200
    if(lvl1['nbsub']>0):
        if(lvl1['nbsub']>1):
            subs = None
            now = lvl1
            while(now['nextsub']>0):
                tmp = hals[now['nextsub']-1]
                subs = tmp if(subs is None) else np.hstack( (subs, tmp) )
                now = tmp
            nproc = min(32, len(subs))
            kwargs = {'rmax_pkpc':2*lvl1['r']/snap.unit['kpc'], 'pos_code':pos_code, 'm_msol':mass_msol}
            with Pool(processes=nproc) as pool:
                results = pool.starmap(calc_virial_mp, [(sub,kwargs) for sub in subs])
        else:
            tmp = hals[lvl1['nextsub']-1]
            r200kpc, m200, r200 = calc_virial(tmp['x'], tmp['y'], tmp['z'], 2*lvl1['r']/snap.unit['kpc'], pos_code, mass_msol)
            virials[tmp['id']-1]['r200kpc'] = r200kpc
            virials[tmp['id']-1]['m200'] = m200
            virials[tmp['id']-1]['r200'] = r200

        for result in results:
            virials[result[0]-1]['r200kpc'] = result[1]
            virials[result[0]-1]['m200'] = result[2]
            virials[result[0]-1]['r200'] = result[3]
pklsave(virials, f"{database}/virial_radius_{mode}_{iout}.pickle")
uri.timer.verbose=1


  0%|          | 0/13706 [00:00<?, ?it/s]

[469] load done


  0%|          | 1/13706 [00:10<41:49:46, 10.99s/it]

[474] load done
    Nbsub: 29


  0%|          | 1/13706 [01:02<236:47:20, 62.20s/it]


AttributeError: Can't pickle local object 'custom_extra_fields.<locals>.<lambda>'

In [10]:
type(pos_code)

numpy.ndarray

In [ ]:
snap_star.get_part(pname='star', target_fields=['x','y','z','m'], nthread=32)


ValueError: could not broadcast input array from shape (0,79020) into shape (0,)

In [ ]:
lvl1s['id']

array([  469,   474,   475, ..., 18772, 18773, 18774], dtype=int32)

In [ ]:
lvl1['id']

474